# Analysing Data with Pandas (cont.)

In this notebook we extract information as we did in the previous part, but now we consider all files and take the mean.

In [1]:
import os
import json
import pandas as pd

In [2]:
def deck_stats(path):
    with open(path) as f:
        data =json.load(f)
    data = data['data']['mainBoard']
    
    deck_cards = pd.json_normalize(data,max_level=1)
    
    db = deck_cards[['types','count']].explode('types').groupby('types',as_index=False).sum()
    
    db1 = deck_cards[ deck_cards.apply(lambda x: False if ' '.join(x.types).count('Land') else True,axis=1) ]  # exclude Land cards
    db1 = db1[['colorIdentity','count']].explode('colorIdentity').fillna('N').groupby('colorIdentity', as_index=False).sum()

    db2 = deck_cards[ deck_cards.apply(lambda x: False if ' '.join(x.types).count('Land') else True,axis=1) ]  # exclude Land cards
    db2 = db2[['manaValue','colorIdentity','count']]
    db2 = db2.explode('colorIdentity').fillna('N').groupby(['manaValue','colorIdentity'], as_index = False).sum()
    f.close()
    return(db,db1,db2)

def is_cmd_deck(path):
    with open(path) as f:
        data =json.load(f)
    data = data['data']['commander']
    f.close()
    return bool(data)

In [3]:
#initalize db

db,db1,db2 = deck_stats('../data/mtg_cards_data/AllDeckFiles/NecronDynasties_40K.json')
db = db.drop(columns ='count')
db1 = db1.drop(columns ='count')
db2 = db2.drop(columns ='count')

In [4]:
directory = '../data/mtg_cards_data/AllDeckFiles'

for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if os.path.isfile(f):
        if is_cmd_deck(f):
            d,d1,d2 = deck_stats(f)
            db = db.merge(d,how='outer', on= 'types',suffixes=('',filename.split('.')[0])).fillna(0)
            db1 = db1.merge(d1,how='outer',on= 'colorIdentity',suffixes=('',filename.split('.')[0])).fillna(0)
            db2 = db2.merge(d2,how='outer',on= ['manaValue','colorIdentity'],suffixes=('',filename.split('.')[0])).fillna(0)

In [5]:
db.set_index('types').mean(axis=1)

types
Artifact        13.243243
Creature        27.522523
Enchantment      5.918919
Instant          7.081081
Land            38.000000
Sorcery          8.747748
Tribal           0.108108
Planeswalker     0.315315
dtype: float64

In [6]:
db1.set_index('colorIdentity').mean(axis=1)

colorIdentity
B    13.207207
N     9.369369
G    14.027027
R    11.279279
U    12.108108
W    12.990991
dtype: float64

In [7]:
db2.drop(columns='colorIdentity').sort_values('manaValue').groupby('manaValue').sum().mean(axis=1)

manaValue
0.0      0.126126
1.0      3.738739
2.0     14.648649
3.0     18.441441
4.0     13.054054
5.0     10.387387
6.0      7.018018
7.0      3.738739
8.0      1.207207
9.0      0.342342
10.0     0.198198
11.0     0.063063
12.0     0.018018
dtype: float64